# NNodely Documentation - Training

In [ ]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj
import os

In [ ]:
vehicle = nnodely(visualizer=MPLVisualizer(),seed=2, workspace=os.path.join(os.getcwd(), 'results'))

# Dimensions of the layers
n  = 25
na = 21

#Create neural model inputs
velocity = State('vel')
brake = Input('brk')
gear = Input('gear')
torque = Input('trq')
altitude = Input('alt',dimensions=na)
acc = Input('acc')

# Create neural network relations
air_drag_force = Linear(b=True)(velocity.last()**2)
breaking_force = -Relu(Fir(W_init = init_negexp, W_init_params={'size_index':0, 'first_value':0.002, 'lambda':3})(brake.sw(n)))
gravity_force = Linear(W_init=init_constant, W_init_params={'value':0}, dropout=0.1, W='gravity')(altitude.last())
fuzzi_gear = Fuzzify(6, range=[2,7], functions='Rectangular')(gear.last())
local_model = LocalModel(input_function=lambda: Fir(W_init = init_negexp, W_init_params={'size_index':0, 'first_value':0.002, 'lambda':3}))
engine_force = local_model(torque.sw(n), fuzzi_gear)

sum_rel = air_drag_force+breaking_force+gravity_force+engine_force

# Closing the loop
sum_rel.closedLoop(velocity)

# Create neural network output
out1 = Output('accelleration', sum_rel)
out2 = Output('velout', altitude.sw(2))

# Add the neural model to the nnodely structure and neuralization of the model
vehicle.addModel('acc',[out1,out2])
vehicle.addMinimize('acc_error', acc.last(), out1, loss_function='rmse')
vehicle.neuralizeModel(0.05)

## Feed-forward Training

Use the 'trainModel' function to train a neuralized model. The function accept various arguments. you can decide which model to train (in case of a multi-model system) and what dataset to use.

The 'splits' argument will use the previously dataset loaded in the framework and divide it in [train, validation, test] according to the percentage

In [ ]:
vehicle.trainModel(models=['acc'], splits=[70,20,10])

you can specify various training parameters such as the shuffling, batch size, learning rate and number of epochs...

In [ ]:
vehicle.trainModel(models=['acc'], train_dataset='dataset_train', train_batch_size=8, val_batch_size=4, test_batch_size=1, num_of_epochs=10, lr=0.001, shuffle_data=True)

## Recurrent Training

The recurrent training can take place only when there are State variables, closed-loops or connections between inputs.

In case of a recurrent training, other parameters can be specified such as the number of prediction window. This is used to select for how many steps doing the recurrent loop

In [ ]:
vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10)

The 'step' size is used to decide whether to skip samples at each epoch. this will ensure a faster training time expecially when the prediction window is too high.

In [ ]:
vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10, step=5)

The 'closed_loop' and 'connect' arguments are used if you want to do a recurrent loop only during training time without having the needs for your network to keep the state of the system inside a State variable

In [ ]:
vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10, step=5, connect={'velout': 'vel'})

In [ ]:
vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10, step=5, closed_loop={'accelleration': 'acc'})

## Set different weight for the minimization functions

use the 'minimize_gain' attribute to modify the importance of certain minimization functions by passing a dictionary with the gain factor for each minimizers.

In [ ]:
vehicle.trainModel(train_dataset='dataset_train', num_of_epochs=10, lr=0.001, shuffle_data=True, minimize_gain={'acc_error':2})

## Early stopping and Best Model

Use one of the built-in early stopping and selection model functions or use a custom one. 

In [ ]:
from nnodely.earlystopping import early_stop_patience, select_best_model
vehicle.trainModel(train_dataset='dataset_train', num_of_epochs=10, lr=0.001, shuffle_data=True, early_stopping=early_stop_patience, select_model=select_best_model)